In [1]:
 #http://localhost:8501/

In [2]:
import os
import json
import pandas as pd

In [3]:
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

#mysql connector to conect with python
mydb = mysql.connector.connect(host = 'localhost', user = 'root', password = '')
mycursor = mydb.cursor(buffered=True,)

#create an engine to insert data value
engine = create_engine("mysql+mysqlconnector://root:@localhost/phonepe")
#connection = engine.connect()

#create database and use for table creataion
mycursor.execute('CREATE DATABASE IF NOT EXISTS phonepe')
mycursor.execute('USE phonepe')

In [6]:
#creating a table
mycursor.execute('''CREATE TABLE IF NOT EXISTS agg_phonpedata_trans(States VARCHAR(100), Years INT, Quarter INT,
                 TransactionName VARCHAR(300), TransactionCount BIGINT, TransactionAmount FLOAT)''')
 
mycursor.execute('''CREATE TABLE IF NOT EXISTS agg_phonepe_user(States VARCHAR(100), Years INT, Quarter INT, Brands VARCHAR(50),
                 UserCount BIGINT, percentage FLOAT)''')

mycursor.execute('''CREATE TABLE IF NOT EXISTS map_trans(States VARCHAR(100), Years INT, Quarter INT, Districts VARCHAR(100),
                 TransactionCount BIGINT, TransactionAmount FLOAT)''')

mycursor.execute('''CREATE TABLE IF NOT EXISTS map_user(States VARCHAR(100), Years INT, Quarter INT, Districts VARCHAR(100),
                 registeredUsers BIGINT, appOpens BIGINT)''')

mycursor.execute('''CREATE TABLE IF NOT EXISTS top_trans(States VARCHAR(100), Years INT, Quarter INT, pincodes INT, TransactionCount BIGINT,
                 TransactionAmount BIGINT)''')

mycursor.execute('''CREATE TABLE IF NOT EXISTS top_user(States VARCHAR(100), Years INT, Quarter INT, pincodes INT,
                 registeredUsers BIGINT)''')

In [5]:
#aggregated_trans

def agg_trans():
    path1 = "C:/Users/kathiravansiva/Desktop/Phonepe_Vis/pulse/data/aggregated/transaction/country/india/state/"
    agg_trans_list = os.listdir(path1)

    columns = {"States":[], "Years" :[], "Quarter":[], "TransactionName":[], "TransactionCount" :[], "TransactionAmount":[] }

    for i in agg_trans_list:
        cur_state = path1 + i + "/"
        agg_year_list = os.listdir(cur_state)

        for j in agg_year_list:
            cur_yrs = cur_state + j + "/"
            agg_file_list = os.listdir(cur_yrs)

            for k in agg_file_list:
                cur_file = cur_yrs + k
                data = open(cur_file,'r')
            
                A = json.load(data)
            
                for z in A["data"]["transactionData"]:
                    name = z['name']
                    count = z['paymentInstruments'][0]['count']
                    amount = z['paymentInstruments'][0]['amount']
                    columns["TransactionName"].append(name)
                    columns["TransactionCount"].append(count)
                    columns["TransactionAmount"].append(amount)
                    columns["States"].append(i)
                    columns["Years"].append(j)
                    columns["Quarter"].append(int(k.strip('.json')))

    phonepe_data = pd.DataFrame(columns)

    #changing States 
    phonepe_data["States"] = phonepe_data["States"].str.replace('andaman-&-nicobar-islands', 'Andaman & Nicobar')
    phonepe_data["States"] = phonepe_data["States"].str.replace('-',' ')
    phonepe_data["States"] = phonepe_data["States"].str.title()
    phonepe_data["States"] = phonepe_data["States"].str.replace('Dadra & Nagar Haveli & Daman & Diu','Dadra and Nagar Haveli and Daman and Diu' )

    return(phonepe_data)

agg_trans().to_sql('agg_phonpedata_trans',engine,if_exists='replace',index = False)
mydb.commit()
            

In [6]:
#aggregated_user

def agg_user_data():
    path2 = "C:/Users/kathiravansiva/Desktop/Phonepe_Vis/pulse/data/aggregated/user/country/india/state/"
    agg_trans_list = os.listdir(path2)

    columns2 = {"States":[], "Years" :[], "Quarter":[], "Brands":[], "UserCount" :[], "percentage":[] }
    
    for i_user in agg_trans_list:
        cur_state1 = path2 + i_user +'/'
        agg_year_list = os.listdir(cur_state1)

        for j_user in agg_year_list:
            cur_yrs1 = cur_state1 + j_user + '/'
            agg_file_list = os.listdir(cur_yrs1)

            for k_user in agg_file_list:
                cur_file1 = cur_yrs1 + k_user
                data1 = open(cur_file1, 'r')

                A2 = json.load(data1)
                try:
                    for z_user in A2["data"]['usersByDevice']:
                        brand = z_user['brand']
                        count = z_user['count']
                        percentage = z_user['percentage']
                        columns2["Brands"].append(brand)
                        columns2["UserCount"].append(count)
                        columns2["percentage"].append(percentage)
                        columns2["States"].append(i_user)
                        columns2["Years"].append(j_user)
                        columns2["Quarter"].append(int(k_user.strip('.json')))
                except:
                    pass
    Phonepe_user = pd.DataFrame(columns2)

    #changing States 
    Phonepe_user["States"] = Phonepe_user["States"].str.replace('andaman-&-nicobar-islands', 'Andaman & Nicobar')
    Phonepe_user["States"] = Phonepe_user["States"].str.replace('-',' ')
    Phonepe_user["States"] = Phonepe_user["States"].str.title()
    Phonepe_user["States"] = Phonepe_user["States"].str.replace('Dadra & Nagar Haveli & Daman & Diu','Dadra and Nagar Haveli and Daman and Diu' )
    return(Phonepe_user)
agg_user_data().to_sql('agg_phonepe_user',engine,if_exists='replace',index = False)
mydb.commit()

In [7]:
#map_trans
def map_trans_data():
    path3 = "C:/Users/kathiravansiva/Desktop/Phonepe_Vis/pulse/data/map/transaction/hover/country/india/state/"
    map_trans_list = os.listdir(path3)

    columns3= {"States":[], "Years" :[], "Quarter":[], "Districts":[], "TransactionCount" :[], "TransactionAmount":[] }

    for map_states in map_trans_list:
        m_state = path3 + map_states + '/'
        map_years_list = os.listdir(m_state)

        for map_years in map_years_list:
            m_years = m_state + map_years + '/'
            map_files_list = os.listdir(m_years)

            for map_files in map_files_list:
                m_files = m_years + map_files
                data2 = open(m_files,'r')

                A3 = json.load(data2)
                for map_tran in A3['data']['hoverDataList']:
                    name = map_tran['name']
                    count = map_tran['metric'][0]['count']
                    amount = map_tran['metric'][0]['amount']
                    columns3['Districts'].append(name)
                    columns3['TransactionCount'].append(count)
                    columns3['TransactionAmount'].append(amount)
                    columns3['States'].append(map_states)
                    columns3['Years'].append(map_years)
                    columns3['Quarter'].append(int(map_files.strip('.json')))

    map_trans_df = pd.DataFrame(columns3)

    #changing States 
    map_trans_df["States"] = map_trans_df["States"].str.replace('andaman-&-nicobar-islands', 'Andaman & Nicobar')
    map_trans_df["States"] = map_trans_df["States"].str.replace('-',' ')
    map_trans_df["States"] = map_trans_df["States"].str.title()
    map_trans_df["States"] = map_trans_df["States"].str.replace('Dadra & Nagar Haveli & Daman & Diu','Dadra and Nagar Haveli and Daman and Diu' )
    
    return(map_trans_df)

map_trans_data().to_sql('map_trans',engine,if_exists='replace',index=False)
mydb.commit()

In [8]:
#map_user
def map_user_data():
    path4 = "C:/Users/kathiravansiva/Desktop/Phonepe_Vis/pulse/data/map/user/hover/country/india/state/"
    map_user_list = os.listdir(path4)

    columns4 = {"States":[], "Years" :[], "Quarter":[], "Districts":[], 'registeredUsers':[], 'appOpens':[] }

    for mu_sta in map_user_list:
        mu_states = path4 + mu_sta +  '/'
        map_uyears_list = os.listdir(mu_states)
        
        for mu_yrs in map_uyears_list:
            mu_years = mu_states + mu_yrs +  '/'
            map_ufiles_list = os.listdir(mu_years)

            for mu_fls in map_ufiles_list:
                mu_files = mu_years + mu_fls
                data3 = open(mu_files,'r')

                A4 = json.load(data3)
                for districts , i in A4['data']['hoverData'].items():
                    regtdusers = i['registeredUsers']
                    appOpens = i['appOpens']
                    columns4['Districts'].append(districts)
                    columns4['registeredUsers'].append(regtdusers)
                    columns4['appOpens'].append(appOpens)
                    columns4['States'].append(mu_sta)
                    columns4['Years'].append(mu_yrs)
                    columns4['Quarter'].append(int(mu_fls.strip('.json')))
    phpe_agUser_Df = pd.DataFrame(columns4)
    #changing States 
    phpe_agUser_Df["States"] = phpe_agUser_Df["States"].str.replace('andaman-&-nicobar-islands', 'Andaman & Nicobar')
    phpe_agUser_Df["States"] = phpe_agUser_Df["States"].str.replace('-',' ')
    phpe_agUser_Df["States"] = phpe_agUser_Df["States"].str.title()
    phpe_agUser_Df["States"] = phpe_agUser_Df["States"].str.replace('Dadra & Nagar Haveli & Daman & Diu','Dadra and Nagar Haveli and Daman and Diu' )
    
    return(phpe_agUser_Df)


map_user_data().to_sql('map_user',engine,if_exists='replace',index=False)
mydb.commit()


In [9]:
#top_trans
def top_trans_data():
    path5 = "C:/Users/kathiravansiva/Desktop/Phonepe_Vis/pulse/data/top/transaction/country/india/state/"
    top_trans_list = os.listdir(path5)

    columns5 = {"States":[], "Years" :[], "Quarter":[], "pincodes":[], 'TransactionCount':[], 'TransactionAmount':[]}

    for top_tsts in top_trans_list:
        tsts = path5 + top_tsts + '/'
        top_years_list = os.listdir(tsts)

        for top_tyrs in top_years_list:
            tyrs = tsts + top_tyrs + '/'
            top_files_list = os.listdir(tyrs)

            for top_tfls in top_files_list:
                tfls = tyrs + top_tfls
                data4 = open(tfls,'r')

                A5 = json.load(data4)
                for top_trans in A5['data']['pincodes']:
                    name = top_trans['entityName']
                    #Type = top_trans['metric']['type']
                    count = top_trans['metric']['count']
                    amount = top_trans['metric']['amount']
                    columns5['States'].append(top_tsts)
                    columns5['Years'].append(top_tyrs)
                    columns5['Quarter'].append(int(top_tfls.strip('.json')))
                    columns5['pincodes'].append(name)
                    #columns5['type'].append(Type)
                    columns5['TransactionCount'].append(count)
                    columns5['TransactionAmount'].append(amount)
    phonepe_Ttop = pd.DataFrame(columns5)
    #changing States 
    phonepe_Ttop["States"] = phonepe_Ttop["States"].str.replace('andaman-&-nicobar-islands', 'Andaman and Nicobar')
    phonepe_Ttop["States"] = phonepe_Ttop["States"].str.replace('-',' ')
    phonepe_Ttop["States"] = phonepe_Ttop["States"].str.title()
    phonepe_Ttop["States"] = phonepe_Ttop["States"].str.replace('Dadra & Nagar Haveli & Daman & Diu','Dadra and Nagar Haveli and Daman and Diu' )
    return(phonepe_Ttop)

top_trans_data().to_sql('top_trans',engine,if_exists='replace',index = False)
mydb.commit()

In [10]:
#top_user
def top_user_data():
    path6 = "C:/Users/kathiravansiva/Desktop/Phonepe_Vis/pulse/data/top/user/country/india/state/"
    top_user_list = os.listdir(path6)

    columns6 = {"States":[], "Years" :[], "Quarter":[], "pincodes":[], 'registeredUsers':[]}

    for top_usts in top_user_list:
        t_usts = path6 + top_usts + '/'
        top_uyrs_list = os.listdir(t_usts)

        for top_uyrs in top_uyrs_list:
            t_uyrs = t_usts + top_uyrs + '/'
            top_ufls_list = os.listdir(t_uyrs)

            for top_ufls in top_ufls_list:
                t_ufls = t_uyrs + top_ufls
                data5 = open(t_ufls,'r' )

                A6 = json.load(data5)
                for top_user in A6['data']['pincodes']:
                    name = top_user['name']
                    regtdusers = top_user['registeredUsers']
                    columns6['States'].append(top_usts)
                    columns6['Years'].append(top_uyrs)
                    columns6['Quarter'].append(int(top_ufls.strip('.json')))
                    columns6['pincodes'].append(name)
                    columns6['registeredUsers'].append(regtdusers)

    phonepe_tuserdf = pd.DataFrame(columns6)
    #changing States 
    phonepe_tuserdf["States"] = phonepe_tuserdf["States"].str.replace('andaman-&-nicobar-islands', 'Andaman and Nicobar')
    phonepe_tuserdf["States"] = phonepe_tuserdf["States"].str.replace('-',' ')
    phonepe_tuserdf["States"] = phonepe_tuserdf["States"].str.title()
    phonepe_tuserdf["States"] = phonepe_tuserdf["States"].str.replace('Dadra & Nagar Haveli & Daman & Diu','Dadra and Nagar Haveli and Daman and Diu' )
    return(phonepe_tuserdf)

top_user_data().to_sql('top_user',engine,if_exists='replace',index = False)
mydb.commit()